# US Labor Dashboard — Project Notebook (Rubric-Aligned)
_Updated 2025-11-15 23:17 UTC_

This notebook provides a **single, clean, inline-documented** workflow to:
- **Collect** labor statistics from the **BLS Public API** (curated series in four sections).
- **Clean & unify** monthly + quarterly data (Q01..Q04 → Mar/Jun/Sep/Dec).
- **Analyze/visualize** with helper functions (YoY, plots).
- **Export** production files for a **Streamlit dashboard** and a **GitHub Action** that appends new releases monthly.

**Rubric alignment** (Econ 8320):
- Includes required series (Nonfarm Payrolls, Unemployment Rate) plus other indicators from proposal.
- Data is **not fetched on dashboard load**; a monthly **GitHub Action** updates `data/bls_timeseries.csv`.
- Provides at least a **year+** of history (we fetch **2006–present** by default).
- Streamlit app + automation files are generated in the **Export** section at the end.

In [ ]:
import os, json, requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

plt.rcParams['figure.figsize'] = (11, 5)
plt.rcParams['axes.grid'] = True
pd.set_option('display.float_format', lambda x: f"{x:,.3f}")
print('Pandas', pd.__version__)

In [ ]:
# =============================
# Configuration with References
# =============================
# Series reflect the proposal categories: Employment, Productivity, Price Index, Compensation.
# CPI is NSA (headline). Productivity series is Q/Q % (PRS85006093). ECI includes both Index (I) and official YoY (A).
START_YEAR = 2006
END_YEAR = datetime.utcnow().year

SERIES = {
    # Employment (Monthly, SA)
    "LNS12000000": {"section": "Employment", "name": "Civilian Employment (Thousands, SA)", "freq": "M"},
    "CES0000000001": {"section": "Employment", "name": "Total Nonfarm Employment (Thousands, SA)", "freq": "M"},
    "LNS14000000": {"section": "Employment", "name": "Unemployment Rate (% SA)", "freq": "M"},
    "CES0500000002": {"section": "Employment", "name": "Avg Weekly Hours, Total Private (SA)", "freq": "M"},
    "CES0500000003": {"section": "Employment", "name": "Avg Hourly Earnings, Total Private ($, SA)", "freq": "M"},
    # Productivity (Quarterly, SA) — Q/Q %
    "PRS85006093": {"section": "Productivity", "name": "Output per Hour — Nonfarm Business (Q/Q %)", "freq": "Q"},
    # Price Index (Monthly, NSA)
    "CUUR0000SA0": {"section": "Price Index", "name": "CPI-U All Items (NSA, 1982–84=100)", "freq": "M"},
    # Compensation (Quarterly, NSA)
    "CIU1010000000000I": {"section": "Compensation", "name": "ECI — Total Compensation, Private (Index, NSA)", "freq": "Q"},
    "CIU1010000000000A": {"section": "Compensation", "name": "ECI — Total Compensation, Private (12m % change, NSA)", "freq": "Q"},
}

BLS_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [ ]:
# ============================
# Fetch & Parse (with docstrings)
# ============================
class BLSError(Exception):
    """BLS API error (HTTP or logical)."""
    pass

def fetch_bls_timeseries(series_ids, start_year, end_year):
    """
    Fetch multiple BLS series using v2 API in a single POST.
    Honors BLS_API_KEY if present for higher limits.
    """
    payload = {"seriesid": series_ids, "startyear": str(start_year), "endyear": str(end_year)}
    key = os.getenv("BLS_API_KEY")
    if key:
        payload["registrationkey"] = key
    r = requests.post(BLS_URL, json=payload, timeout=60)
    if r.status_code != 200:
        raise BLSError(f"HTTP {r.status_code}: {r.text[:200]}")
    data = r.json()
    if data.get("status") != "REQUEST_SUCCEEDED":
        raise BLSError(f"BLS error: {json.dumps(data)[:300]}")
    return data

def _q_to_month(q: int) -> int:
    """Quarter → quarter-ending month mapping: Q1→3, Q2→6, Q3→9, Q4→12."""
    return {1: 3, 2: 6, 3: 9, 4: 12}[q]

def series_payload_to_rows(series_json):
    """
    Convert a single BLS 'series' JSON block to tidy rows.
    Accepts monthly (M01..M12) and quarterly (Q01..Q04). Skips M13 (annual avg).
    """
    sid = series_json["seriesID"]
    rows = []
    for item in series_json["data"]:
        p = item.get("period")
        if not p or p == "M13":
            continue
        year = int(item["year"])
        if p.startswith("M"):
            month = int(p[1:])
        elif p.startswith("Q"):
            month = _q_to_month(int(p[1:]))
        else:
            continue
        dt = pd.Timestamp(year=year, month=month, day=1)
        val = float(item["value"])
        rows.append({"series_id": sid, "date": dt, "value": val})
    return rows

In [ ]:
# =============================
# Build the long-form DataFrame
# =============================
api = fetch_bls_timeseries(list(SERIES.keys()), START_YEAR, END_YEAR)
rows = [r for s in api["Results"]["series"] for r in series_payload_to_rows(s)]
df = pd.DataFrame(rows).sort_values(["series_id", "date"]).reset_index(drop=True)
print("Rows:", len(df), "Series:", df["series_id"].nunique())
df.head()

In [ ]:
# ======================
# Transform & Plot Utils
# ======================
def _lag_for_yoy(sid: str) -> int:
    freq = SERIES.get(sid, {}).get("freq", "M").upper()
    return 4 if freq.startswith("Q") else 12

def yoy_from_level(df_long: pd.DataFrame, sid: str) -> pd.DataFrame:
    d = (df_long[df_long.series_id == sid]
         .set_index("date")
         .sort_index()[["value"]]
         .copy())
    lag = _lag_for_yoy(sid)
    d["yoy_pct"] = d["value"].pct_change(lag) * 100.0
    return d

def plot_series(df_long: pd.DataFrame, series_ids, title=None, ylabel=None, since=None):
    sub = df_long[df_long.series_id.isin(series_ids)].copy()
    if since is not None:
        sub = sub[sub.date >= pd.to_datetime(since)]
    name_map = {sid: SERIES.get(sid, {}).get("name", sid) for sid in series_ids}
    for sid in series_ids:
        d = sub[sub.series_id == sid].copy()
        plt.figure()
        plt.plot(d["date"], d["value"])
        ttl = title if (title and len(series_ids) == 1) else name_map.get(sid, sid)
        plt.title(ttl)
        plt.xlabel("Date")
        plt.ylabel(ylabel or "Value")
        plt.show()

def plot_yoy_from_level(df_long: pd.DataFrame, sid: str, title_suffix=" (YoY %)"):
    d = yoy_from_level(df_long, sid)
    plt.figure()
    plt.plot(d.index, d["yoy_pct"])
    plt.title(SERIES.get(sid, {}).get("name", sid) + title_suffix)
    plt.xlabel("Date")
    plt.ylabel("YoY %")
    plt.show()

In [ ]:
# ======================
# Example Charts (run)
# ======================
# Employment
plot_series(df, ["LNS12000000", "CES0000000001", "LNS14000000", "CES0500000002", "CES0500000003"], since=f"{START_YEAR}-01-01")

# Wages YoY
plot_yoy_from_level(df, "CES0500000003")

# Productivity (Q/Q %)
plot_series(df, ["PRS85006093"], ylabel="Q/Q %", title="Output per Hour — Nonfarm Business (Q/Q %)", since=f"{START_YEAR}-01-01")

# CPI level & YoY
plot_series(df, ["CUUR0000SA0"], ylabel="Index", title="CPI-U All Items (NSA, 1982–84=100)", since=f"{START_YEAR}-01-01")
plot_yoy_from_level(df, "CUUR0000SA0")

# ECI: official YoY & YoY-from-index
plot_series(df, ["CIU1010000000000A"], ylabel="%", title="ECI — Total Compensation, Private (12m % change, NSA)", since=f"{START_YEAR}-01-01")
plot_yoy_from_level(df, "CIU1010000000000I", title_suffix=" (YoY from Index)")

## Export: generate Streamlit app, updater script, GitHub Action, requirements
Run the next cell once to write production files into the working directory.
- `bls_update.py` (API → CSV, incremental appender)
- `streamlit_app.py` (interactive dashboard with filters & recession shading)
- `.github/workflows/update.yml` (twice-monthly schedule)
- `requirements.txt`

In [ ]:
from pathlib import Path

DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True, parents=True)

bls_update_py = r"""#!/usr/bin/env python3
import os, json, requests
from datetime import datetime
from pathlib import Path
import pandas as pd

START_YEAR = 2006
END_YEAR = datetime.utcnow().year

SERIES = {
    "LNS12000000": {"section": "Employment", "name": "Civilian Employment (Thousands, SA)", "freq": "M"},
    "CES0000000001": {"section": "Employment", "name": "Total Nonfarm Employment (Thousands, SA)", "freq": "M"},
    "LNS14000000": {"section": "Employment", "name": "Unemployment Rate (% SA)", "freq": "M"},
    "CES0500000002": {"section": "Employment", "name": "Avg Weekly Hours, Total Private (SA)", "freq": "M"},
    "CES0500000003": {"section": "Employment", "name": "Avg Hourly Earnings, Total Private ($, SA)", "freq": "M"},
    "PRS85006093": {"section": "Productivity", "name": "Output per Hour — Nonfarm Business (Q/Q %)", "freq": "Q"},
    "CUUR0000SA0": {"section": "Price Index", "name": "CPI-U All Items (NSA, 1982–84=100)", "freq": "M"},
    "CIU1010000000000I": {"section": "Compensation", "name": "ECI — Total Compensation, Private (Index, NSA)", "freq": "Q"},
    "CIU1010000000000A": {"section": "Compensation", "name": "ECI — Total Compensation, Private (12m % change, NSA)", "freq": "Q"},
}

BLS_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
DATA_DIR = Path("data")
CSV_PATH = DATA_DIR / "bls_timeseries.csv"
META_PATH = DATA_DIR / "meta.json"
DATA_DIR.mkdir(parents=True, exist_ok=True)

class BLSError(Exception): pass

def fetch_bls_timeseries(series_ids, start_year, end_year):
    payload = {"seriesid": series_ids, "startyear": str(start_year), "endyear": str(end_year)}
    key = os.getenv("BLS_API_KEY")
    if key:
        payload["registrationkey"] = key
    r = requests.post(BLS_URL, json=payload, timeout=60)
    if r.status_code != 200:
        raise BLSError(f"HTTP {r.status_code}: {r.text[:200]}")
    data = r.json()
    if data.get("status") != "REQUEST_SUCCEEDED":
        raise BLSError(f"BLS error: {json.dumps(data)[:300]}")
    return data

def _q_to_month(q: int) -> int:
    return {1: 3, 2: 6, 3: 9, 4: 12}[q]

def series_payload_to_rows(series_json):
    sid = series_json["seriesID"]
    rows = []
    for item in series_json["data"]:
        p = item.get("period")
        if not p or p == "M13":
            continue
        year = int(item["year"])
        if p.startswith("M"):
            month = int(p[1:])
        elif p.startswith("Q"):
            month = _q_to_month(int(p[1:]))
        else:
            continue
        dt = pd.Timestamp(year=year, month=month, day=1)
        val = float(item["value"])
        rows.append({"series_id": sid, "date": dt, "value": val})
    return rows

def load_existing():
    if CSV_PATH.exists():
        return pd.read_csv(CSV_PATH, parse_dates=["date"])
    return pd.DataFrame(columns=["series_id", "date", "value"])

def union_and_dedupe(df_old, df_new):
    df = pd.concat([df_old, df_new], ignore_index=True)
    df = df.drop_duplicates(subset=["series_id", "date"], keep="last")
    return df.sort_values(["series_id", "date"]).reset_index(drop=True)

def run_full_or_incremental():
    df_old = load_existing()
    start = START_YEAR if df_old.empty else max(START_YEAR, (df_old['date'].max() - pd.DateOffset(months=24)).year)
    api = fetch_bls_timeseries(list(SERIES.keys()), start, END_YEAR)
    rows = [r for s in api["Results"]["series"] for r in series_payload_to_rows(s)]
    df_new = pd.DataFrame(rows)
    df_out = union_and_dedupe(df_old, df_new)
    df_out.to_csv(CSV_PATH, index=False)
    META_PATH.write_text(json.dumps({"last_updated_utc": datetime.utcnow().isoformat()}, indent=2))
    print(f"Updated {len(df_out)} rows → {CSV_PATH}")
    return df_out

if __name__ == "__main__":
    run_full_or_incremental()
"""

streamlit_app_py = r"""import json
from pathlib import Path
import pandas as pd
import streamlit as st
import plotly.express as px
from datetime import datetime

DATA_DIR = Path("data")
CSV_PATH = DATA_DIR / "bls_timeseries.csv"
META_PATH = DATA_DIR / "meta.json"

SERIES = {
    "LNS12000000": {"section": "Employment", "name": "Civilian Employment (Thousands, SA)", "freq": "M"},
    "CES0000000001": {"section": "Employment", "name": "Total Nonfarm Employment (Thousands, SA)", "freq": "M"},
    "LNS14000000": {"section": "Employment", "name": "Unemployment Rate (% SA)", "freq": "M"},
    "CES0500000002": {"section": "Employment", "name": "Avg Weekly Hours, Total Private (SA)", "freq": "M"},
    "CES0500000003": {"section": "Employment", "name": "Avg Hourly Earnings, Total Private ($, SA)", "freq": "M"},
    "PRS85006093": {"section": "Productivity", "name": "Output per Hour — Nonfarm Business (Q/Q %)", "freq": "Q"},
    "CUUR0000SA0": {"section": "Price Index", "name": "CPI-U All Items (NSA, 1982–84=100)", "freq": "M"},
    "CIU1010000000000I": {"section": "Compensation", "name": "ECI — Total Compensation, Private (Index, NSA)", "freq": "Q"},
    "CIU1010000000000A": {"section": "Compensation", "name": "ECI — Total Compensation, Private (12m % change, NSA)", "freq": "Q"},
}

SECTIONS = ["Employment", "Productivity", "Price Index", "Compensation"]
RECESSIONS = [
    (pd.Timestamp(2007, 12, 1), pd.Timestamp(2009, 6, 1)),
    (pd.Timestamp(2020, 2, 1), pd.Timestamp(2020, 4, 1)),
]

@st.cache_data
def load_data():
    df = pd.read_csv(CSV_PATH, parse_dates=["date"])
    df["series_id"] = df["series_id"].astype("string")
    return df

def yoy_from_level(df, sid):
    freq = SERIES.get(sid, {}).get("freq", "M").upper()
    lag = 4 if freq.startswith("Q") else 12
    d = df[df.series_id == sid][["date", "value"]].sort_values("date").set_index("date").copy()
    d["YoY %"] = d["value"].pct_change(lag) * 100.0
    d = d.reset_index()
    d["series_id"] = sid
    return d

def add_recession_shading(fig):
    for (start, end) in RECESSIONS:
        fig.add_vrect(x0=start, x1=end, fillcolor="gray", opacity=0.15, line_width=0)
    return fig

def main():
    st.set_page_config(page_title="US Labor Dashboard", layout="wide")
    st.title("US Labor Dashboard")
    st.caption("Auto-updating BLS dashboard (Econ 8320 project)")

    with st.expander("About & rubric alignment", expanded=False):
        st.markdown(
            "- Uses BLS Public API via monthly/quarterly fetcher (stored to CSV; no live fetch on every app load).\n"
            "- Includes required series: Nonfarm Employment & Unemployment Rate; plus additional sections from proposal.\n"
            "- Updates via GitHub Actions twice monthly to catch major releases.\n"
            "- Filter by section/series and date range; optional YoY visuals for CPI, Wages, ECI index."
        )

    if META_PATH.exists():
        meta = json.loads(META_PATH.read_text())
        st.caption(f"Last updated (UTC): {meta.get('last_updated_utc', 'unknown')}")

    section = st.sidebar.multiselect("Sections", SECTIONS, default=SECTIONS)
    eligible = [sid for sid, m in SERIES.items() if m["section"] in section]
    pick = st.sidebar.multiselect(
        "Series",
        eligible,
        format_func=lambda x: f"{SERIES[x]['section']} — {SERIES[x]['name']}",
        default=eligible,
    )
    year_min, year_max = st.sidebar.slider("Year range", 2006, datetime.utcnow().year, (2006, datetime.utcnow().year))

    if not CSV_PATH.exists():
        st.error("Data file not found. Run bls_update.py first.")
        return
    df = load_data()
    df = df[df["series_id"].isin(pick)]
    df = df[(df["date"].dt.year >= year_min) & (df["date"].dt.year <= year_max)]

    st.download_button("Download full CSV", CSV_PATH.read_bytes(), file_name="bls_timeseries.csv")
    st.download_button("Download filtered CSV", df.to_csv(index=False).encode("utf-8"), file_name="bls_timeseries_filtered.csv")

    for sec in SECTIONS:
        sub_ids = [sid for sid in pick if SERIES[sid]["section"] == sec]
        if not sub_ids:
            continue
        st.subheader(sec)
        for sid in sub_ids:
            name = SERIES[sid]["name"]
            d = df[df.series_id == sid].sort_values("date")
            if d.empty: continue
            fig = px.line(d, x="date", y="value", title=name, labels={"value": "Value", "date": "Date"})
            fig = add_recession_shading(fig)
            st.plotly_chart(fig, use_container_width=True)

            if sid in ["CUUR0000SA0", "CES0500000003", "CIU1010000000000I"]:
                yoy = yoy_from_level(df, sid).dropna()
                if not yoy.empty:
                    fig2 = px.line(yoy, x="date", y="YoY %", title=f"{name} — YoY %")
                    fig2 = add_recession_shading(fig2)
                    st.plotly_chart(fig2, use_container_width=True)

    st.write("---")
    st.caption("Notes: CPI is NSA; productivity series is Q/Q %; ECI shown as official YoY and YoY computed from the index.")

if __name__ == "__main__":
    main()
"""

workflow_yml = r"""name: Update BLS data monthly
on:
  workflow_dispatch: {}
  schedule:
    - cron: "0 10 5 * *"
    - cron: "0 10 15 * *"
jobs:
  update:
    runs-on: ubuntu-latest
    permissions:
      contents: write
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.11"
      - name: Install deps
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
      - name: Update BLS data
        env:
          BLS_API_KEY: ${{ secrets.BLS_API_KEY }}
        run: python bls_update.py
      - name: Commit & push
        run: |
          git config user.name "github-actions[bot]"
          git config user.email "github-actions[bot]@users.noreply.github.com"
          git add data/bls_timeseries.csv data/meta.json
          git commit -m "ci: monthly BLS data update [skip ci]" || echo "No changes"
          git push
"""

requirements_txt = "pandas>=2.1
requests>=2.31
streamlit>=1.36
plotly>=5.20
"

# Write files
Path("bls_update.py").write_text(bls_update_py)
Path("streamlit_app.py").write_text(streamlit_app_py)
Path(".github/workflows").mkdir(parents=True, exist_ok=True)
Path(".github/workflows/update.yml").write_text(workflow_yml)
Path("requirements.txt").write_text(requirements_txt)

print("Exported: bls_update.py, streamlit_app.py, .github/workflows/update.yml, requirements.txt")